<a href="https://colab.research.google.com/github/KorStats/skku/blob/main/code/%EC%9A%94%EC%95%BD%EB%AC%B8_%EC%A6%9D%EA%B0%95.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 필요 라이브러리 설치

In [1]:
pip install transformers sentencepiece torch rouge_score

In [2]:
!pip install konlpy
!pip install textrankr
!pip install JPype1>=0.7.0

# 요약문 알고리즘

## ① TextRank

In [3]:
import re
import torch
import numpy as np
import kss
import networkx as nx
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from transformers import PreTrainedTokenizerFast, BartForConditionalGeneration, T5Tokenizer, T5ForConditionalGeneration, BertTokenizer, BertForSequenceClassification
from rouge_score import rouge_scorer


# TextRank 기반 추출적 요약
def textrank_summary(text, ratio=0.2):
    sentences = text.split('. ')# 문장 단위로 분리
    vectorizer = TfidfVectorizer()
    sentence_vectors = vectorizer.fit_transform(sentences)
    similarity_matrix = cosine_similarity(sentence_vectors, sentence_vectors)
    graph = nx.from_numpy_array(similarity_matrix)
    scores = nx.pagerank(graph)
    ranked_sentences = sorted(((scores[i], s) for i, s in enumerate(sentences)), reverse=True)
    top_n = max(1, int(len(sentences) * ratio))  # 최소 1문장은 포함
    summarized_sentences = [ranked_sentences[i][1] for i in range(top_n)]
    return ' '.join(summarized_sentences)

## ② KoBART

In [4]:
!git clone https://github.com/seujung/KoBART-summarization.git

fatal: destination path 'KoBART-summarization' already exists and is not an empty directory.


In [5]:
!pip install loguru
!pip install lightning

In [6]:
cd /content/drive/MyDrive/KoBART-summarization

[Errno 2] No such file or directory: '/content/drive/MyDrive/KoBART-summarization'
/content


In [7]:
!pip install -r requirements.txt
!pip install torch==1.7.1+cu101 torchvision==0.8.2+cu101 -f https://download.pytorch.org/whl/torch_stable.html

ERROR: Could not open requirements file: [Errno 2] No such file or directory: 'requirements.txt'
Looking in links: https://download.pytorch.org/whl/torch_stable.html
ERROR: Could not find a version that satisfies the requirement torch==1.7.1+cu101 (from versions: 1.13.0, 1.13.0+cpu, 1.13.0+cu116, 1.13.0+cu117, 1.13.0+cu117.with.pypi.cudnn, 1.13.1, 1.13.1+cpu, 1.13.1+cu116, 1.13.1+cu117, 1.13.1+cu117.with.pypi.cudnn, 2.0.0, 2.0.0+cpu, 2.0.0+cpu.cxx11.abi, 2.0.0+cu117, 2.0.0+cu117.with.pypi.cudnn, 2.0.0+cu118, 2.0.1, 2.0.1+cpu, 2.0.1+cpu.cxx11.abi, 2.0.1+cu117, 2.0.1+cu117.with.pypi.cudnn, 2.0.1+cu118, 2.0.1+rocm5.3, 2.0.1+rocm5.4.2, 2.1.0, 2.1.0+cpu, 2.1.0+cpu.cxx11.abi, 2.1.0+cu118, 2.1.0+cu121, 2.1.0+cu121.with.pypi.cudnn, 2.1.0+rocm5.5, 2.1.0+rocm5.6, 2.1.1, 2.1.1+cpu, 2.1.1+cpu.cxx11.abi, 2.1.1+cu118, 2.1.1+cu121, 2.1.1+cu121.with.pypi.cudnn, 2.1.1+rocm5.5, 2.1.1+rocm5.6, 2.1.2, 2.1.2+cpu, 2.1.2+cpu.cxx11.abi, 2.1.2+cu118, 2.1.2+cu121, 2.1.2+cu121.with.pypi.cudnn, 2.1.2+rocm5.5, 2.1

In [8]:
# 클론한 파일 제공 데이터
%cd /content/KoBART-summarization/data
!tar -zxvf train.tar.gz
!tar -zxvf test.tar.gz
!ls

/content/KoBART-summarization/data
train.tsv
test.tsv
신문기사_요약학습데이터.csv  dain_train.csv  test.tar.gz  test.tsv  train.tar.gz  train.tsv


In [7]:
# 추가할 데이터를 현재 커널로 가져온다.
import pandas as pd
import shutil
# train
shutil.copy("/content/KoBART-summarization/data/신문기사_요약학습데이터.csv", "/content/KoBART-summarization/data/dain_train.csv")
df_train = pd.read_csv('/content/KoBART-summarization/data/dain_train.csv', encoding='utf-8-sig', on_bad_lines='skip')
df_train = df_train.dropna(axis=0)
df_train.to_csv("/content/KoBART-summarization/data/train.tsv", mode='a', header=0, sep='\t', index=False)

# valid
#shutil.copy("/content/KoBART-summarization/data/신문기사_요약학습데이터.csv", "/content/KoBART-summarization/data/dain_test.csv")
#df_valid = pd.read_csv('/content/KoBART-summarization/data/dain_train.csv', encoding='utf-8-sig', on_bad_lines='skip')
#df_valid = df_valid.dropna(axis=0)
#df_valid.to_csv("/content/KoBART-summarization/data/test.tsv", mode='a', header=0, sep='\t', index=False)

In [10]:
#재학습
%cd /content/KoBART-summarization
!python train.py --gradient_clip_val 1.0 --max_epochs 1 --default_root_dir logs --gpus 1 --batch_size 10 --num_workers 4
#python train.py  --gradient_clip_val 1.0 --max_epochs 50 --default_root_dir logs  --batch_size 4 --num_workers 4  : cpu 재학습

/content/KoBART-summarization
2025-01-30 03:15:29.195815: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1738206929.573600    3359 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1738206929.687830    3359 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-01-30 03:15:30.443839: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
tokenizer.json: 100% 682k/682k [00:00<00:00, 5.13MB/s]
added_tokens.json: 100% 4.00/4.00 [00:00

In [12]:
# Get Model
# kobart_summary 디렉토리에 모델 저장
# hparams: logs 하위 디렉토리에서 사용할 모델의 버전 골라 hparams.yaml set
# model_binary: logs 하위 디렉토리에서 사용할 체크포인트 골라 *.ckpt set
!python get_model_binary.py --hparams ./logs/tb_logs/default/version_0/hparams.yaml --model_binary ./logs/model_chp/epoch=00-val_loss=1.464.ckpt

2025-01-30 03:16:10.150513: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1738206970.190633    3652 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1738206970.229147    3652 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-01-30 03:16:10.272030: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
Traceback (most recent call last):
  File "/content/KoBART-summarization/get_model_binary.py", line 12, in <module>
    inf =

In [9]:
#Test
!pip install git+https://github.com/SKT-AI/KoBART

  Cloning https://github.com/SKT-AI/KoBART to /tmp/pip-req-build-xj0ukz78
  Running command git clone --filter=blob:none --quiet https://github.com/SKT-AI/KoBART /tmp/pip-req-build-xj0ukz78
  Resolved https://github.com/SKT-AI/KoBART to commit eec563bfccf723cae8fd0fff02d5b2b09e847516
  Preparing metadata (setup.py) ... done
  Using cached boto3-1.36.9-py3-none-any.whl.metadata (6.6 kB)
  Using cached pytorch_lightning-1.2.1-py3-none-any.whl.metadata (25 kB)
INFO: pip is looking at multiple versions of kobart to determine which version is compatible with other requirements. This could take a while.
ERROR: Could not find a version that satisfies the requirement torch==1.7.1 (from kobart) (from versions: 1.13.0, 1.13.1, 2.0.0, 2.0.1, 2.1.0, 2.1.1, 2.1.2, 2.2.0, 2.2.1, 2.2.2, 2.3.0, 2.3.1, 2.4.0, 2.4.1, 2.5.0, 2.5.1, 2.6.0)
ERROR: No matching distribution found for torch==1.7.1


In [10]:
import torch
from transformers import PreTrainedTokenizerFast
from transformers import BartForConditionalGeneration

tokenizer = PreTrainedTokenizerFast.from_pretrained('digit82/kobart-summarization')
model = BartForConditionalGeneration.from_pretrained('digit82/kobart-summarization')

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.


In [11]:
# KoBART 기반 추상적 요약
def Kobart_summary(text):
    input_ids = tokenizer.encode(text)
    input_ids = torch.tensor(input_ids)
    input_ids = input_ids.unsqueeze(0)
    output = model.generate(input_ids, eos_token_id=1, max_length=512, num_beams=5)
    output = tokenizer.decode(output[0], skip_special_tokens=True)
    return output

# 데이터 불러오기

In [32]:
from google.colab import drive
drive.mount('/content/gdrive')
!ln -s /content/gdrive/My\ Drive/ /mydrive
!ls /mydrive

Mounted at /content/gdrive
'Colab Notebooks'   법률_train_original.csv	  신문기사_valid_original.csv
 kosbi_train.json   법률_valid_original.csv	  이전
'My Drive'	    신문기사_train_original.csv   졸업논문


In [33]:
import pandas as pd
df=pd.read_csv('/content/gdrive/MyDrive/졸업논문/DATA/예시데이터.csv', encoding='utf-8-sig')

In [34]:
df['text']=df['context']+df['sentence']
df_1=df[df['sentence_label']=='safe']
df_2=df[df['sentence_label']=='unsafe']

In [35]:
df_2.columns

Index(['demographic_category', 'demographic_group', 'context', 'context_en',
       'sentence', 'sentence_en', 'context_label', 'context_sub_label',
       'sentence_label', 'sentence_sub_label', 'annotated_demo', 'text'],
      dtype='object')

In [36]:
df_2['summary_ext'] = df_2['text'].apply(textrank_summary)
df_2['summary_abs'] = df_2['text'].apply(Kobart_summary)

<ipython-input-36-4c59339f50ce>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_2['summary_ext'] = df_2['text'].apply(textrank_summary)
<ipython-input-36-4c59339f50ce>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_2['summary_abs'] = df_2['text'].apply(Kobart_summary)


In [38]:
df_2[['text', 'summary_ext', 'summary_abs']].head(10)

,text,summary_ext,summary_abs
3,저는 아직까지 결혼하지 않은 30대 초반의 남성입니다.주변에서도 그렇고 제 스스로도...,저는 아직까지 결혼하지 않은 30대 초반의 남성입니다.주변에서도 그렇고 제 스스로도...,결혼하지 않은 30대 초반의 남성인 30대 초반의 남성인 저는 결혼하지 않은 30대...
4,밀레니얼 세대는 인터넷과 IT 기술에 능통하다.그래서 이들을 겨냥한 마케팅 전략이 ...,밀레니얼 세대는 인터넷과 IT 기술에 능통하다.그래서 이들을 겨냥한 마케팅 전략이 ...,밀레니얼 세대는 인터넷과 IT 기술에 능통한 밀레니얼 세대는 인터넷과 IT 기술에 ...
5,우리 엄마는 아빠와 이혼해서 나와 동생을 키우셨다.지금 생각해보면 그때 어떻게 살았...,우리 엄마는 아빠와 이혼해서 나와 동생을 키우셨다.지금 생각해보면 그때 어떻게 살았...,우리 엄마는 아빠와 이혼해서 나와 동생을 키우셨는데 그때 어떻게 살았는지 모르겠지만...
6,문과 계열 학과로 진학하면 취직하기가 어렵다.문과는 취업률이 낮아서 이과 쪽으로 진...,문과 계열 학과로 진학하면 취직하기가 어렵다.문과는 취업률이 낮아서 이과 쪽으로 진...,취업률이 낮아서 이과 쪽으로 진로를 바꾸는 학생들이 많다고 하더라.문과는 취업률이 ...


In [ ]:
df_2.to_csv('요약문증강.csv', encoding='utf-8-sig')

# 요약문 평가
- ROUGE (Recall-Oriented Understudy for Gisting Evaluation) 점수는 자동 요약 및 자연어 생성 모델의 성능을 평가하는 대표적인 방법 중 하나입니다. 주요 ROUGE 지표는 다음과 같습니다.
- ROUGE-1: 요약된 문장에서 정답과 일치하는 단어 단위(n=1)의 개수를 기반으로 측정
- ROUGE-2: 정답과 요약이 공유하는 2-그램(n=2) 단위의 개수 기반 평가
- ROUGE-L: 정답과 요약된 문장 간 가장 긴 공통 부분 문자열(LCS, Longest Common Subsequence) 기반 평가

In [43]:
df=df_2

In [45]:
df.columns

Index(['demographic_category', 'demographic_group', 'context', 'context_en',
       'sentence', 'sentence_en', 'context_label', 'context_sub_label',
       'sentence_label', 'sentence_sub_label', 'annotated_demo', 'text',
       'summary_ext', 'summary_abs'],
      dtype='object')

In [46]:
from rouge_score import rouge_scorer
import pandas as pd

# ROUGE 점수 계산 함수
def calculate_rouge(df, summary_col, reference_col):
    scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
    scores = {'rouge1': [], 'rouge2': [], 'rougeL': []}
    for ref, pred in zip(df[reference_col], df[summary_col]):
        score = scorer.score(ref, pred)
        scores['rouge1'].append(score['rouge1'].fmeasure)
        scores['rouge2'].append(score['rouge2'].fmeasure)
        scores['rougeL'].append(score['rougeL'].fmeasure)
    return {metric: sum(values)/len(values) for metric, values in scores.items()}

# 평가 수행
rouge_scores_abs = calculate_rouge(df, 'summary_abs', 'text') #정답데이터 : summary (=text)
rouge_scores_ext = calculate_rouge(df, 'summary_ext', 'text') #정답데이터 : summary (=text)

# 결과 출력
print("ROUGE 점수 (KoBART 요약)", rouge_scores_abs)
print("ROUGE 점수 (TextRank 요약)", rouge_scores_ext)


ROUGE 점수 (KoBART 요약) {'rouge1': 0.15833333333333333, 'rouge2': 0.0, 'rougeL': 0.15833333333333333}
ROUGE 점수 (TextRank 요약) {'rouge1': 0.5, 'rouge2': 0.0, 'rougeL': 0.5}
